# Pathfinder for Hidden connections 

Some text

In [1]:
#Imports
from neo4j import GraphDatabase, basic_auth
import neo4jupyter
import pandas as pd
import sys  
sys.path.insert(0, '')
from vis_class import vis_class

In [2]:
#Create db connector
login = open("Login.txt", "r")
uri = login.readline()
user = login.readline()
password = ""
login.close()

driver = GraphDatabase.driver(uri, auth=basic_auth(user, password))
session = driver.session(database="neo4j")

## General Path finding / matching

CASE n.eyes
  WHEN 'blue'  THEN 1
  WHEN 'brown' THEN 2
  ELSE 3
END AS result

In [8]:
#https://neo4j.com/docs/cypher-manual/current/introduction/uniqueness/ 
#https://stackoverflow.com/questions/14345555/return-unique-nodes-in-cypher-path-query
#https://github.com/neo4j/neo4j/issues/12628
#How to improve it to just get once Episode I: The Phantom Menace?
five_path = '''MATCH p = ()-[*5]-() RETURN nodes(p) Limit 5'''
results = session.read_transaction(
    lambda tx: tx.run(five_path).data())
results

[{'nodes(p)': [{'name': 'Episode I: The Phantom Menace'},
   {'name': 'JAR JAR'},
   {'name': 'JABBA'},
   {'name': 'FODE/BEED'},
   {'name': 'Episode I: The Phantom Menace'},
   {'name': 'BRAVO THREE'}]},
 {'nodes(p)': [{'name': 'Episode I: The Phantom Menace'},
   {'name': 'JAR JAR'},
   {'name': 'JABBA'},
   {'name': 'FODE/BEED'},
   {'name': 'Episode I: The Phantom Menace'},
   {'name': 'GREEDO'}]},
 {'nodes(p)': [{'name': 'Episode I: The Phantom Menace'},
   {'name': 'JAR JAR'},
   {'name': 'JABBA'},
   {'name': 'FODE/BEED'},
   {'name': 'Episode I: The Phantom Menace'},
   {'name': 'OBI-WAN'}]},
 {'nodes(p)': [{'name': 'Episode I: The Phantom Menace'},
   {'name': 'JAR JAR'},
   {'name': 'JABBA'},
   {'name': 'FODE/BEED'},
   {'name': 'Episode I: The Phantom Menace'},
   {'name': 'EMPEROR'}]},
 {'nodes(p)': [{'name': 'Episode I: The Phantom Menace'},
   {'name': 'JAR JAR'},
   {'name': 'JABBA'},
   {'name': 'FODE/BEED'},
   {'name': 'Episode I: The Phantom Menace'},
   {'name': '

## Finding hidden path

Something like that should it be
match (m:Movie) 
where m.title="Episode VI: Return of the Jedi" 
match (p:Person) 
where p.title="KI-ADI-MUNDI" 
match  
searchPath=(Movie)<-[*]-(Person)-[*]->(Movie)-[*]-(Person)-[*]->(Person) 
return  distinct m.title,searchPath 

In [ ]:
#Find path between Person and Movie

In [ ]:
#Find path between Person and Person which are not related at the first glance

In [ ]:
#Find path between Movie and Movie which are not related at the first glance

## In which movies are same charactes

## Closing the connection to the database

In [3]:
#Close all connections
driver.close()
session.close()
drive = None
session = None